In [1]:
import io
import sqlite3
import pandas as pd

In [2]:
#Vocês devem executar o script do outro notebook chamado PopulandoBD: https://github.com/Pendulun/TP2-IBD/blob/master/PopulandoBD.ipynb
conn = sqlite3.connect('C:/Users/Daniel/Desktop/BDpopulado/database.db') # Esse é o BD. Ele vai ser gerado automaticamente no mesmo diretório que salvarem esse .ipynb
cursor = conn.cursor()

In [3]:
#Esse comando PRAGMA está exemplificado na especificação do tp pelo professor, acredito que seja assim que executa
cursor.execute("PRAGMA cache_size = 0")

Como combinado, cada um deve pensar em pelo menos 4 consultas sendo que cada consulta deve ser feita de duas formas diferentes para testes de eficiência. Além disso, tentem fazer consultas que suprimam as necessidades das especificações:
- 2 consultas envolvendo as operações de seleção e projeção;
- 3 consultas envolvendo a junção de duas relações;
- 3 consultas envolvendo a junção de três ou mais relações;
- 2 consultas envolvendo funções de agregação sobre o resultado da junção de pelo menos duas relações.

Obviamente, se cada um precisa fazer apenas 4 consultas, não será possível cada um fazer as 10 consultas necessárias, porém, se tentarem bolar 1 consulta por requisito já fica ótimo.

In [15]:
#Esse comando é para ver o tempo que leva para executar a query. Ela também é especificada pelo professor.
#%timeit

In [26]:
%timeit pd.read_sql_query("SELECT NUM_OCORRENCIAS, COUNT(opepesid) FROM (SELECT COUNT(opepesid) AS NUM_OCORRENCIAS, opepesid FROM OCORRENCIAPESSOA GROUP BY opepesid) GROUP BY NUM_OCORRENCIAS",conn)

9.17 s ± 1.14 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
df=pd.read_sql_query("SELECT NUM_OCORRENCIAS, COUNT(opepesid) FROM (SELECT COUNT(opepesid) AS NUM_OCORRENCIAS, opepesid FROM OCORRENCIAPESSOA GROUP BY opepesid) GROUP BY NUM_OCORRENCIAS",conn)
df

,NUM_OCORRENCIAS,COUNT(opepesid)
0,1,3464148
1,2,32558
2,3,6
3,4,62


In [4]:
%timeit pd.read_sql_query("SELECT COUNT(pesid), NUMOCORRENCIAS FROM (SELECT pesid, COUNT(opeid) AS NUMOCORRENCIAS FROM (PESSOA JOIN OCORRENCIAPESSOA ON pesid=opepesid) GROUP BY pesid ) GROUP BY NUMOCORRENCIAS",conn)

39.1 s ± 1.81 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
df = pd.read_sql_query("SELECT COUNT(pesid), NUMOCORRENCIAS FROM( SELECT pesid, COUNT(opeid)"+
                       " AS NUMOCORRENCIAS FROM (PESSOA JOIN OCORRENCIAPESSOA ON pesid=opepesid) GROUP BY pesid )"+
                       " GROUP BY NUMOCORRENCIAS",conn)
df

,COUNT(pesid),NUMOCORRENCIAS
0,3464118,1
1,32558,2
2,6,3
3,62,4


In [8]:
%timeit pd.read_sql_query("SELECT ttadescricao, COUNT(ttacodigo) AS NUM_OCORRENCIAS FROM (OCORRENCIAACIDENTE JOIN TIPOACIDENTE ON ttacodigo = oacttacodigo) GROUP BY ttadescricao ORDER BY NUM_OCORRENCIAS DESC",conn)

1.86 s ± 195 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
df = pd.read_sql_query("SELECT ttadescricao, COUNT(ttacodigo) AS NUM_OCORRENCIAS FROM (OCORRENCIAACIDENTE JOIN"+
                       " TIPOACIDENTE ON ttacodigo = oacttacodigo) GROUP BY ttadescricao ORDER BY NUM_OCORRENCIAS DESC",conn)
df

,ttadescricao,NUM_OCORRENCIAS
0,Colisão traseira,306958
1,Colisão lateral,181130
2,Saída de Pista,157776
3,Colisão Transversal,104993
4,Colisão com objeto fixo,65679
5,Capotamento,51433
6,Tombamento,42925
7,Colisão frontal,37006
8,Queda de motocicleta / bicicleta / veículo,31794
9,Atropelamento de pessoa,31208


In [15]:
%timeit pd.read_sql_query("SELECT ttadescricao, COUNT(ttacodigo) AS NUM_OCORRENCIAS FROM OCORRENCIAACIDENTE, TIPOACIDENTE WHERE ttacodigo = oacttacodigo GROUP BY ttadescricao ORDER BY NUM_OCORRENCIAS DESC",conn)

1.8 s ± 189 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
df = pd.read_sql_query("SELECT ttadescricao, COUNT(ttacodigo) AS NUM_OCORRENCIAS FROM OCORRENCIAACIDENTE, TIPOACIDENTE WHERE ttacodigo = oacttacodigo GROUP BY ttadescricao ORDER BY NUM_OCORRENCIAS DESC",conn)
df

,ttadescricao,NUM_OCORRENCIAS
0,Colisão traseira,306958
1,Colisão lateral,181130
2,Saída de Pista,157776
3,Colisão Transversal,104993
4,Colisão com objeto fixo,65679
5,Capotamento,51433
6,Tombamento,42925
7,Colisão frontal,37006
8,Queda de motocicleta / bicicleta / veículo,31794
9,Atropelamento de pessoa,31208


In [19]:
%timeit pd.read_sql_query("SELECT COUNT(ocoid) AS NUM_OCORRENCIAS, tufdenominacao FROM ( OCORRENCIA JOIN MUNICIPIO ON ocomunicipio = tmucodigo JOIN UF ON tmuuf = tufuf) GROUP BY tufdenominacao ORDER BY NUM_OCORRENCIAS DESC",conn)

13.7 s ± 1.32 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
df = pd.read_sql_query("SELECT COUNT(ocoid) AS NUM_OCORRENCIAS, tufdenominacao FROM "+
                       "( OCORRENCIA JOIN MUNICIPIO ON ocomunicipio = tmucodigo JOIN UF ON tmuuf = tufuf)"+
                       " GROUP BY tufdenominacao ORDER BY NUM_OCORRENCIAS DESC",conn)
df

,NUM_OCORRENCIAS,tufdenominacao
0,161552,Minas Gerais
1,113616,Santa Catarina
2,105990,Paraná
3,97991,Rio de Janeiro
4,84204,Rio Grande do Sul
5,83176,São Paulo
6,58213,Bahia
7,46576,Espírito Santo
8,45911,Goiás
9,42643,Pernambuco


In [ ]:
%timeit df = pd.read_sql_query("Consulta 3, versão 2",conn)
df

In [21]:
%timeit pd.read_sql_query("SELECT unisigla, unidenominacao FROM UNIDADEOPERACIONAL WHERE unihelicoptero='S'",conn)

1.75 ms ± 64 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [22]:
df = pd.read_sql_query("SELECT unisigla, unidenominacao FROM UNIDADEOPERACIONAL WHERE unihelicoptero='S'",conn)
df

,unisigla,unidenominacao
0,9ªSR/RS,9ª SUPERINTENDÊNCIA REGIONAL DE POLÍCIA RODOVI...
1,3ªSR/MS,DEL.01/PST.03 - Terenos
2,3ªSR/MS,DEL.02 - Delegacia de Nova Alvorada do Sul
3,3ªSR/MS,DEL.02/PST.01 - Nova Alvorada do Sul
4,21ªSR/RO,DEL.01 - Porto Velho
5,21ªSR/RO,DEL.02 - Ji-Paraná
6,21ªSR/RO,DEL.03 - Ariquemes/RO
7,21ªSR/RO,DEL.04 - Vilhena
8,21ªSR/RO,DEL.05 - Rio Branco
9,21ªSR/RO,DEL.05/PST.01 - Posto de Rio Branco/ AC 01


In [ ]:
%timeit df = pd.read_sql_query("Consulta 4, versão 2",conn)
df